In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 4s (61.2 kB/s)
Reading package li

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [4]:
# Drop null values
review_df = df.dropna()
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [5]:
# Create vine table for analysis
vine_df = review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

## Deliverable 2: Determine Basis of Vine Reviews

In [6]:
# Filter data and create new df to retrieve all rows where total_votes >= 20
from pyspark.sql.functions import col
votes_df = vine_df.filter(col("total_votes") >= 20)
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [7]:
# Retrieve all rows where the number of helpful votes divided by total_votes is >= 50%
percent_df = votes_df.filter((col('helpful_votes')/col('total_votes') >= 0.5))
percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [8]:
# Filter df to retrieve all rows where a review was written as part of the vine program
paid_df = percent_df.filter(col('vine') == 'Y')
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N|
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|
| RF2RNZEJO0K1N|          4|           23|         26|   Y|                N|
| ROB6VOW41E8O5|          4|          155|        172|   Y|                N|
|R3ASMXPEPYY28T|          3|           11|         20|   Y|     

In [9]:
# Filter df to retrieve all rows where a review was NOT written as part of the vine program
unpaid_df = percent_df.filter(col('vine') == 'N')
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [10]:
# Determine total number of reviews
from pyspark.sql.functions import col, when, count, lit
ratings_df = percent_df.groupBy("vine").agg(count(col("vine")).alias("Total Reviews"), count(when(col("star_rating") == 5, True)).alias("5 Star Reviews"),
                                           (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("5 Star Percentage"))
ratings_df.show()

+----+-------------+--------------+-----------------+
|vine|Total Reviews|5 Star Reviews|5 Star Percentage|
+----+-------------+--------------+-----------------+
|   Y|         1080|           454|42.03703703703704|
|   N|        49659|         23034|46.38434120703196|
+----+-------------+--------------+-----------------+

